In [2]:
import trimesh
import numpy as np
from sklearn.cluster import KMeans
from collections import deque
import os
from sphereGenUtils import *
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
robotName = "panda"
baseFolder = os.path.abspath(os.path.join(os.path.abspath(""), "../../"))
folder = os.path.abspath(os.path.join(baseFolder,"models/franka_description/meshes/Simplified/watertight"))
files = ["link0.obj", "link1.obj", "link2.obj", "link3.obj", "link4.obj", "link5.obj", "link6.obj", "link7_nohand.obj"]
meshFile = os.path.join(folder, files[3])

In [4]:
mesh = trimesh.load(meshFile)
mesh.show()

In [5]:
# v, f = trimesh.remesh.subdivide_to_size(vertices = mesh.vertices, faces = mesh.faces, max_edge=0.05)
# mesh_ = trimesh.Trimesh(vertices=v, faces=f)
# trimesh.repair.fix_winding(mesh_)
# mesh_.show()

In [6]:
leaf_spheres, root = generateSphereEnvelop(mesh, KDTreeDepth=4)

In [7]:
def count_leafs(node: KDNode) -> int:
    if node.left is None and node.right is None:
        return 1
    count = 0
    if node.left:
        count += count_leafs(node.left)
    if node.right:
        count += count_leafs(node.right)
    return count

# gather points and group points by leaves
def collect_leaf_points(node: KDNode) -> list[np.ndarray]:
    if node.left is None and node.right is None:
        return [node.points]
    points = []
    if node.left:
        points.extend(collect_leaf_points(node.left))
    if node.right:
        points.extend(collect_leaf_points(node.right))
    return points

In [8]:
pts = collect_leaf_points(root)
len(pts)

16

In [9]:
# display the leaf points in different colors
colormap = np.random.uniform(size=(len(pts), 3))

# stack points and colors for visualization
all_points = np.vstack(pts)
all_colors = np.vstack([colormap[i] * np.ones((len(pts[i]), 3)) for i in range(len(pts))])

# create a point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(all_points)
pcd.colors = o3d.utility.Vector3dVector(all_colors)

# Visualize with larger point size
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(pcd)

# Increase point size
opt = vis.get_render_option()
opt.point_size = 5.0  # Adjust this for larger points

# add the mesh
# read the mesh in open3d
mesh_o3d = o3d.io.read_triangle_mesh(meshFile)

# add the mesh
vis.add_geometry(mesh_o3d)

vis.run()
vis.destroy_window()

In [10]:
# === 3. Compute a Bounding Sphere for Each Region ===
# For each region, we compute a sphere that conservatively encloses the vertices.
spheres = []
for i, (c, r) in enumerate(leaf_spheres):
    # Create a sphere mesh for visualization using trimesh's icosphere.
    sphere_mesh = trimesh.creation.icosphere(subdivisions=3, radius=r)
    sphere_mesh.apply_translation(c)
    
    # Optionally set a semi-transparent color (RGBA) for the sphere.
    sphere_mesh.visual.face_colors = [200, 50, 50, 100]
    spheres.append(sphere_mesh) 

# --- 5. Create a Scene and Visualize ---
# Create a scene and add both the original mesh and the computed spheres.
scene = trimesh.Scene()
scene.add_geometry(mesh)
for s in spheres:
    scene.add_geometry(s)

# Optionally, you can adjust the visual appearance of the original mesh
mesh.visual.face_colors = [200, 200, 200, 150]  # Light gray with some transparency

# Display the scene in an interactive window
scene.show()